## Лабораторная работа №2
### Решение СЛАУ прямым и итерацинным методом

$$\begin{equation*}
 \begin{cases}
   a_{11}x_1 + a_{12}x_2 + ... +a_{1n}x_n = f_1, 
   \\
   ~~~~~~~~~~~~~~~~~~~~~~~\dots
   \\
   a_{n1}x_1 + a_{n2}x_2 + ... +a_{nn}x_n = f_n 

\end{cases}
\end{equation*}$$

#### В качестве матрицы используется $A = [a_{ij}]$ и вектор правой части $f = [f_i]$ из задания II.10.5 (л):
$$n = 20; \hspace{1mm} a_{ii} = 10; \hspace{1mm} a_{ij} = \dfrac{1}{i + j}, \hspace{1mm} i \neq j; \hspace{1mm} f_i = \dfrac{1}{i}$$


#### Если матрица А имеет ненулевые главные миноры, то для прямого метода выбирается LU, в противном случае - метод Гаусса с выбором главного элемента. В качестве итерационного метода используется метод верхней релаксации с параметром $w = 1.1$.

#### Максимальное по модулю собственное значение матрицы A рассчитывается итерационным методом:
$$Ax_k = λ_{max} x_{k+1}$$


$$|λ_{max}| = \lim_{k -> \inf}\dfrac{||Ax_k||}{||x_{k+1}||}$$
#### Аналогично рассчитывается $|λ_{min}(A)| = \frac{1}{|λ_{max}(A^{-1})|}$.

#### Для матрицы A рассчитывается число обусловленности $μ_3 = ||A||\cdot||A^{-1}||$ в третьей норме. В данной задаче $μ << 100$, значит задача хорошо обусловленна и можно использовать итерационный метод.

### Результаты вычислений приведены ниже:
```
Eigenvalues: (min|λ(A)|, max|λ(A)|) = (9.657797124947598, 11.300128398425878)
Condition number in 3rd norm: μ = ||A||*||A^(-1)||: 1.1700523682813633
Stop criteria: |Ax - f| < ε = 1e-15

Roots LU method:
 [0.09615746 0.04478375 0.02874755 0.02101576 0.0164943  0.01353947
 0.01146294 0.00992677 0.00874602 0.00781115 0.00705327 0.00642689
 0.0059008  0.00545292 0.00506717 0.00473155 0.00443698 0.00417641
 0.00394434 0.00373636]
Solution Discrepancy:  1.9040911533613541e-16

Roots upper relaxation method:
 [0.09615746 0.04478375 0.02874755 0.02101576 0.0164943  0.01353947
 0.01146294 0.00992677 0.00874602 0.00781115 0.00705327 0.00642689
 0.0059008  0.00545292 0.00506717 0.00473155 0.00443698 0.00417641
 0.00394434 0.00373636]
Solution Discrepancy:  2.4970378841646907e-16

Discrepancy between directive and iterative methods: 4.3246485775520446e-17
```

In [22]:
import numpy as np

n = 20
epsilon = 1e-15
w = 1.1

def CreateEmptyMatrix():
    return np.zeros((n, n))

def CreateRandomVec():
    return np.array(np.random.rand(n))

def InitMatA():
    mat_A = CreateEmptyMatrix()

    for i in range(0, n):
        for j in range(0, n):
            if i == j:
                mat_A[i][j] = 10
            else:
                mat_A[i][j] = 1 / (i + j + 2)

    return mat_A

def InitVecF():
    return np.array([1 / (i + 1) for i in range(0, n)])

In [23]:
def SolutionDiscrepancy(mat_A, vec_x, vec_F):
    return np.linalg.norm(vec_F - np.matmul(mat_A, vec_x))

def ComputeEigenvalues(mat_A):
    eig, _ = np.linalg.eig(mat_A)
    return eig.min(), eig.max()

def ConditionNumber(mat_A):
    eig_min, eig_max = ComputeEigenvalues(mat_A)
    return np.abs(eig_max / eig_min)

In [24]:
def SelectLeadingElem(mat_A, vec_F, j, selected_range):
    for i in selected_range:
        if mat_A[i][j] > mat_A[j][j]:
            mat_A[i], mat_A[j] = mat_A[j], mat_A[i]
            vec_F[i], vec_F[j] = vec_F[j], vec_F[i]

def GaussCourse(mat_A, vec_F, j, selected_range):
    for i in selected_range:
        vec_F[i] -= vec_F[j] * (mat_A[i][j] / mat_A[j][j])
        mat_A[i] -= mat_A[j] * (mat_A[i][j] / mat_A[j][j])

    vec_F[j] /= mat_A[j][j]
    mat_A[j] /= mat_A[j][j]

def DirectCourse(mat_A, vec_F):
    for j in range(0, n):
        SelectLeadingElem(mat_A, vec_F, j, range(j + 1, n))
        GaussCourse(mat_A, vec_F, j, range(j + 1, n))

def ReverseCourse(mat_A, vec_F):
    for j in reversed(range(0, n)):
        SelectLeadingElem(mat_A, vec_F, j, reversed(range(0, j)))
        GaussCourse(mat_A, vec_F, j, reversed(range(0, j)))

def GaussMethod(mat_A, vec_F):
    local_mat_A = mat_A.copy()
    local_vec_F = vec_F.copy()

    DirectCourse(local_mat_A, local_vec_F)
    ReverseCourse(local_mat_A, local_vec_F)
    return local_vec_F

In [25]:
def IsLUDecomposable(mat_A):
    for i in range(0, n):
        mat_i = np.array([lines[0: i + 1] for lines in mat_A[0: i + 1]])
        if not np.linalg.det(mat_i):
            return False

    return True

def DecomposeLU(mat_A):
    mat_L = CreateEmptyMatrix()
    mat_U = CreateEmptyMatrix()

    for i in range(0, n):
        mat_L[i][i] = 1

    for i in range(0, n):
        for j in range(0, n):
            summ = 0
            if i <= j:
                for k in range(0, i):
                    summ += mat_L[i][k] * mat_U[k][j]
                mat_U[i][j] = mat_A[i][j] - summ
            if i > j:
                for k in range(0, j):
                    summ += mat_L[i][k] * mat_U[k][j]
                mat_L[i][j] = (mat_A[i][j] - summ) / mat_U[j][j]

    return mat_L, mat_U

def LUMethod(mat_A, vec_F):
    if not IsLUDecomposable(mat_A):
        return GaussMethod(mat_A, vec_F)

    local_vec_F = vec_F.copy()
    mat_L, mat_U = DecomposeLU(mat_A)
    DirectCourse(mat_L, local_vec_F)
    ReverseCourse(mat_U, local_vec_F)    
    return local_vec_F

In [26]:
def InitRelaxationMethod(mat_A, vec_F):
    mat_L = CreateEmptyMatrix()
    mat_D = CreateEmptyMatrix()
    mat_U = CreateEmptyMatrix()

    for i in range(0, n):
        mat_D[i][i] = mat_A[i][i]
        for j in range(i + 1, n):
            mat_U[i][j] = mat_A[i][j]
            mat_L[j][i] = mat_A[j][i]

    mat_inv = np.linalg.inv(mat_D + w * mat_L)
    mat_B = - np.matmul(mat_inv, (w - 1) * mat_D + w * mat_U)
    vec_f = np.matmul(w * mat_inv, vec_F)
    return mat_B, vec_f

def UpperRelaxationMethod(mat_A, vec_F):
    mat_B, vec_f = InitRelaxationMethod(mat_A, vec_F)
    vec_x = CreateRandomVec()

    while SolutionDiscrepancy(mat_A, vec_x, vec_F) > epsilon:
        vec_x = np.matmul(mat_B, vec_x) + vec_f

    return vec_x

In [27]:
def main():
    mat_A = InitMatA()
    vec_F = InitVecF()

    print("Eigenvalues: (min|λ(A)|, max|λ(A)|) =", ComputeEigenvalues(mat_A))
    print("Condition number in 3rd norm: μ = ||A||*||A^(-1)||:", ConditionNumber(mat_A))
    print("Stop criteria: |Ax - f| < ε =", epsilon)

    vec_x = LUMethod(mat_A, vec_F)
    print("\nRoots LU method:\n", vec_x)
    print("Solution Discrepancy: ", SolutionDiscrepancy(mat_A, vec_x, vec_F))

    vec_y = UpperRelaxationMethod(mat_A, vec_F)
    print("\nRoots upper relaxation method:\n", vec_y)
    print("Solution Discrepancy: ", SolutionDiscrepancy(mat_A, vec_y, vec_F))
    print("\nDiscrepancy between directive and iterative methods:", np.linalg.norm(vec_x - vec_y))

main()

Eigenvalues: (min|λ(A)|, max|λ(A)|) = (9.657797124947598, 11.300128398425878)
Condition number in 3rd norm: μ = ||A||*||A^(-1)||: 1.1700523682813633
Stop criteria: |Ax - f| < ε = 1e-15

Roots LU method:
 [0.09615746 0.04478375 0.02874755 0.02101576 0.0164943  0.01353947
 0.01146294 0.00992677 0.00874602 0.00781115 0.00705327 0.00642689
 0.0059008  0.00545292 0.00506717 0.00473155 0.00443698 0.00417641
 0.00394434 0.00373636]
Solution Discrepancy:  1.9040911533613541e-16

Roots upper relaxation method:
 [0.09615746 0.04478375 0.02874755 0.02101576 0.0164943  0.01353947
 0.01146294 0.00992677 0.00874602 0.00781115 0.00705327 0.00642689
 0.0059008  0.00545292 0.00506717 0.00473155 0.00443698 0.00417641
 0.00394434 0.00373636]
Solution Discrepancy:  2.4970378841646907e-16

Discrepancy between directive and iterative methods: 4.3246485775520446e-17
